In [ ]:
## 加载必备库文件

import numpy as np

import mxnet as mx
from mxnet import nd
from mxnet import gluon
from mxnet import autograd

from mxnet.gluon import nn
from mxnet.gluon import data as gdata
from mxnet.gluon.data import DataLoader
from mxnet.gluon.data.vision import transforms as gtransforms

from gluoncv import model_zoo

In [ ]:
## 步骤
# 1. 读取样本和标注
# 1. transform
# 1. mini-batch 样本迭代器
# 1. 定义 loss 以及 metric
# 1. 定义网络结构并初始化权重
# 1. 确定最优化方法
# 1. train

In [ ]:
## 损失函数
l1_loss = gluon.loss.L1Loss()
sigmoid_ce = gluon.loss.SigmoidBinaryCrossEntropyLoss(from_sigmoid=False)
# 第一个参数是 pred ，第二个参数是 label
softmax_ce = gluon.loss.SoftmaxCrossEntropyLoss()

In [ ]:
## 性能度量函数
def metric_val(net, dataset_val, metric=None, ctx=mx.cpu(0)):
    
    if metric is None:
        metric = mx.metric.Accuracy()
        
    for _, batch in enumerate(dataset_val):
        # 将数据切片分别加载到不同的设备上
        data_val = gluon.utils.split_and_load(batch[0], ctx, batch_axis=0)
        label_val = gluon.utils.split_and_load(batch[1], ctx, batch_axis=0)
        
        yhat = [net(x) for x in data_val]
        
        # 第一个参数是 label ， 第二个参数是 output
        metric.update(label_val, yhat)    
  
    return metric.get()

In [ ]:
from gluoncv.utils.metrics.coco_detection import COCODetectionMetric
COCODetectionMetric??

In [2]:
from gluoncv.data.transforms import presets
presets.yolo.YOLO3DefaultTrainTransform??
presets.rcnn.FasterRCNNDefaultTrainTransform??


In [ ]:
## 数据预处理
## 牢记 mxnet 使用 BCHW 形式

# Dataset 类提供了两个转换函数： transform_first 和 transform ；
# transform_first 只变换 data ； transform 同时变换样本和标签（一个样本的所有数据）

# imgnet 数据集
mean = [0.485, 0.456, 0.406]
std  = [0.229, 0.224, 0.225]

# 写两个函数，一个是 train_transform 一个 test_transform
def train_transform(img, label, mean, std):
    """
    YOLOv3 默认的数据预处理，图像和标签都需要处理
    1. random color jittering
    1. random expansion with prob 0.5
    1. random cropping
    1. resize with random interpolation
    1. random horizontal flip
    1. to tensor
    1. nomalize
    """
    
    # random color jittering
    mx.nd.image.random_color_jitter()
    
    # random expansion with prob 0.5
    
    # random cropping
    mx.image.fixed_crop()
    mx.image.random_crop()
    
    # resize with random interpolation
    mx.image.imresize() # interp 利用入参进行了设置 (img, width, height, interp=interp)
    
    # random horizontal flip
    mx.nd.image.random_flip_left_right()
    
    # to tensor WHC -> CHW
    img = mx.nd.image.to_tensor(img)
    
    # nomalize
    img = mx.nd.image.normalize(img, mean, std)
    
    return img, label

def test_transform(img, lable, mean, std):
    
    # resize with random interpolation
    mx.image.imresize() # interp 利用入参进行了设置 (img, width, height, interp=interp)
    
    # to tensor WHC -> CHW
    img = mx.nd.image.to_tensor(img)
    
    # nomalize
    img = mx.nd.image.normalize(img, mean, std)
    
    return img, label


# gluoncv 代码实现
def transform(src, label, width, height, mean, std):
    """Apply transform to training image/label."""
    # random color jittering
    img = experimental.image.random_color_distort(src)

    # random expansion with prob 0.5
    if np.random.uniform(0, 1) > 0.5:
        img, expand = timage.random_expand(img, fill=[m * 255 for m in mean])
        bbox = tbbox.translate(label, x_offset=expand[0], y_offset=expand[1])
    else:
        img, bbox = img, label

    # random cropping
    h, w, _ = img.shape
    bbox, crop = experimental.bbox.random_crop_with_constraints(bbox, (w, h))
    x0, y0, w, h = crop
    img = mx.image.fixed_crop(img, x0, y0, w, h)

    # resize with random interpolation
    h, w, _ = img.shape
    interp = np.random.randint(0, 5)
    img = timage.imresize(img, width, height, interp=interp)
    bbox = tbbox.resize(bbox, (w, h), (width, height))

    # random horizontal flip
    h, w, _ = img.shape
    img, flips = timage.random_flip(img, px=0.5)
    bbox = tbbox.flip(bbox, (w, h), flip_x=flips[0])

    # to tensor
    img = mx.nd.image.to_tensor(img)
    img = mx.nd.image.normalize(img, mean=mean, std=std)

    return img, bbox.astype(img.dtype)

In [ ]:
## 数据加载
# from mxnet.gluon.data import DataLoader
# 这个函数最主要实现的目的就是每次返回 batch_size 大小的样本 [ Loads data from a dataset and returns mini-batches of data. ]
# 阅读 DataLoader 代码发现，这个函数就是先依据入参生成所有样本 batch_sampler (经过 shuffle 或者顺序读取)，
# 然后生成一个迭代器，每次返回 batch_size 大小的样本，且会将这些样本进行函数 batchify_fn 处理 
# 当然还可以使用多个线程同时读取，可以预先读取一定数量的样本等等
# 可以自己去实现，但是感觉没有必要
# 参数
#    dataset : ndarray or numpy array
#    batch_size : int
#    shuffle : bool
#    sampler : Sampler
#    last_batch : {'keep', 'discard', 'rollover'}
#    batch_sampler : Sampler
#    batchify_fn : callable. 用户自定义组装样本的方法
#    num_workers : int, default 0. 使用 num_workers 个线程来读取样本
#    pin_memory : boolean, default False. 使用函数 mxnet.ndarray.ndarray.NDArray.as_in_context(context) 实现，加快从 CPU 到 GPU 的拷贝速度
#    prefetch : int, default is `num_workers * 2`. 预处理样本的个数，会消耗较大的 shared_memory （应该是 GPU 的），当 num_workers > 0 时生效

loader_train = DataLoader(voc_train.transform(transforms_train), 
                              batch_size=batch_size, 
                              shuffle=True, 
                              last_batch='discard', 
                              num_workers=num_workers)

loader_val = DataLoader(voc_val.transform(transforms_val),
                            batch_size=batch_size,
                            shuffle=False, 
                            num_workers=num_workers)

for ib, batch in enumerate(loader_train):
    if ib > 0:
        break
    print('data:', batch[0][0].shape)
    print('label:', batch[6][0].shape)
    # 将数据切片分别加载到不同的设备上 ； 始终卡在这里，下一条打印信息没有出来
    data_train = gluon.utils.split_and_load(batch, ctx_list=ctx, batch_axis=0)
    
    print("aaa")

    with autograd.record():
        input_order = [0, 6, 1, 2, 3, 4, 5]
        obj_loss, center_loss, scale_loss, cls_loss = yolov3_model(*[data_train[o] for o in input_order])
        # sum up the losses
        # some standard gluon training steps:
        # autograd.backward(sum_loss)
        # trainer.step(batch_size)
        print("bbb")

In [3]:
from mxnet.gluon.data import DataLoader
DataLoader??

In [ ]:
# 学习参数
# 调节学习速率衰减的倍数
lr_decay = 0.1
lr_decay_epochs_set = set([200, 400])

def update_learn_rate(trainer, epoch, lr_decay_epochs_set, lr_decay=0.1):
    if epoch in lr_decay_epochs_set:
        trainer.set_learning_rate(trainer.learning_rate * lr_decay)


from gluoncv.utils import TrainingHistory
train_history = TrainingHistory(['train', 'val'])


# 最优化
optimizer = mx.optimizer.Adam(learning_rate=0.0001,
                             beta1=0.9,
                             beta2=0.999,
                             epsilon=1e-08,
                             lazy_update=True)

trainer = gluon.Trainer(yolov3_model.collect_params(), optimizer)



In [5]:
from mxnet import gluon
gluon.Trainer??

In [ ]:
## 冻结原有层的权重
#resnet18_cifar10[0].collect_params().setattr('grad_req', 'null')
## 初始化自定义层的权重
#resnet18_cifar10[1].initialize(init=mx.init.Xavier(), ctx=ctx)

In [ ]:
epochs = 100

## 网络训练
def train_net(net, data_train, data_val, trainer, epochs):
    
    for epoch in range(epochs):
        
        #metric.reset()
        train_loss = 0
        update_learn_rate(trainer, epoch, lr_decay_epochs, lr_decay)
        tic = time.time()
        
        # dataset 是可迭代对象
        for i, batch in enumerate(loader_train):
            # 将数据切片分别加载到不同的设备上；这里假如有 n 个 GPU ，前面的 DataLoader 中 batch_size 是不是应该是 batch*n ? TODO
            data_train = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0)
            label_train = gluon.utils.split_and_load(batch[1], ctx_list=ctx, batch_axis=0)
            
            # 自动求导。record() 函数使得 mxnet 记录并计算梯度，需要训练的参数都需要计算梯度
            with autograd.record():
                output = [net(X) for X in data_train]
                loss = [loss_fn(yhat, y) for yhat, y in zip(output, label_train)]

            # loss 是 list ，当有多个 loss 的时候，所有的 loss 都需要反向传播
            # l 是 mxnet.ndarray.ndarray.NDArray 格式的数据，本身就有 backward() 函数
            # 调用 backward() 函数用于计算梯度
            for l in loss:
                l.backward()
            
            # 更新参数， 通过调用 allreduce_grads() 和 update() 来实现参数的更新
            # 必须在 autograd.backward() 之后，以及 record() 之外调用
            # allreduce_grads() 必须在 trainer.update() 之前调用
            # 这里更新的时候是怎样用到上面计算的 loss 的？
            # trainer 已经中指定了需要训练的参数，应该是可以在某个位置找到这些参数的梯度，从而使用指定的最优化方法来更新参数
            trainer.step(batch_size)
                        
            for l in loss:
                train_loss += l.sum().asscalar() / batch_size
            
            # 每个 batch 更新一下训练的准确率
            metric.update(label_train, output)
            
        _, acc = metric.get()
        _, val_acc = metric_val(net, dataset_val, ctx=ctx)
        
        # 这里记录的是错误率
        train_history.update([1-acc, 1-val_acc])
        
        toc = time.time()
        
        print('[epoch %d] train_loss=%f, acc=%f, val_acc=%f, lr=%.9f, time: %fs' % 
              (epoch, train_loss, acc, val_acc, trainer.learning_rate, toc-tic))